## Install (Colab Only)
Skip this step when running locally.

In [1]:
# install
!pip install pyepo
!pip install gurobipy
!pip install mpax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s 

Remove the problematic hook in Google Colab

In [1]:
import sys
sys.meta_path = [hook for hook in sys.meta_path if not any(keyword in str(hook) for keyword in ["google.colab"])]

# Optimization Model

PyEPO is an implementation that aims to support an end-to-end predict-then-optimize with linear objective function and unknown cost coefficients. The core component is the differentiable optimization solver, which is involved in updating the gradient of the cost coefficients with respect to the optimal solution.

For ease of use, the implementation extends PyTorch's automatic function to construct the solver. As a result, users have the flexibility to select the solvers and algorithms and subsequently obtain the optimal solution while simultaneously computing the gradient.

This tutorial will provide examples of building optimization models ``optModel`` with PyEPO.

## 1 Problem Example: Shortest Path

Consider a 5x5 grid network, represented as a weighted graph. The figure shows that each node has top, bottom, left, and right neighbors. We aim to find the shortest path from left top to right bottom.

<img src="https://github.com/khalil-research/PyEPO/blob/main/images/shortestpath.png?raw=1" width="500">

The weighted graph includes 25 nodes and 40 edges. The weights of the edges are the costs of the path. Let's set random weights.

In [1]:
import random
# random seed
random.seed(42)
# set random cost for test
cost = [random.random() for _ in range(40)]

## 2 Introducation to optModel

``optModel`` is a module of PyEPO library. It is not a solver but serves as a container of a solver or an algorithm. This design allows for flexibility in the selection of solvers and algorithms by users. ``optModel`` treats these solvers as black boxes and provides interfaces ``_getModel``, ``setObj``, and ``solve``. Other modules of PyEPO can use ``optModel`` for tasks such as training and testing.

Methods of ``optModel``:
- ``_getModel``: Build and return optimization solver and corresponding decision variables.
- ``setObj``: Give a cost vector to set the objective function.
- ``solve``: Solve optimization problem and return optimal solution and objective value.

### 2.1 Build Model with NetworkX and Dijkstra Agorithm

We can build the graph with NetworkX and uses Dijkstra’s method to compute the shortest weighted path between two nodes in a graph.

用 i * edge + j 对node和edge进行编码

In [21]:
import networkx as nx
import numpy as np
from pyepo.model.opt import optModel

class myShortestPathModel1(optModel):

    def __init__(self, grid):
        """
        Args:
            grid (tuple): size of grid network
        """
        self.grid = grid
        self.arcs = self._getArcs()
        super().__init__()

    def _getModel(self):
        """
        A method to build model

        Returns:
            tuple: optimization model and variables
        """
        # build graph as optimization model
        g = nx.Graph()
        # add arcs as variables
        g.add_edges_from(self.arcs, cost=0)
        return g, g.edges

    def setObj(self, c):
        """
        A method to set objective function

        Args:
            c (ndarray): cost of objective function
        """
        # set weights for edges
        for i, e in enumerate(self.arcs):
            self._model.edges[e]["cost"] = c[i]

    def solve(self):
        """
        A method to solve model

        Returns:
            tuple: optimal solution (list) and objective value (float)
        """
        # dijkstra
        path = nx.shortest_path(self._model, weight="cost", source=0, target=self.grid[0]*self.grid[1]-1)
        # convert path into active edges
        edges = []
        u = 0
        for v in path[1:]:
            edges.append((u,v))
            u = v
        # init sol & obj
        sol = np.zeros(self.num_cost)
        obj = 0
        # convert active edges into solution and obj
        for i, e in enumerate(self.arcs):
            if e in edges:
                sol[i] = 1 # active edge
                obj += self._model.edges[e]["cost"] # cost of active edge
        return sol, obj

    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges on columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

In [22]:
# solve model
optmodel = myShortestPathModel1(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 2.2869938328922332
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


In [27]:
help(optmodel.solve)

Help on method solve in module __main__:

solve() method of __main__.myShortestPathModel1 instance
    A method to solve model

    Returns:
        tuple: optimal solution (list) and objective value (float)



In [26]:
optmodel.__dir__()

['grid',
 'arcs',
 'modelSense',
 '_model',
 'x',
 '__module__',
 '__init__',
 '_getModel',
 'setObj',
 'solve',
 '_getArcs',
 '__doc__',
 '__abstractmethods__',
 '_abc_impl',
 '__repr__',
 'num_cost',
 'copy',
 'addConstr',
 'relax',
 '__dict__',
 '__weakref__',
 '__slots__',
 '__new__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [31]:
optmodel.arcs

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (0, 5),
 (1, 6),
 (2, 7),
 (3, 8),
 (4, 9),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (5, 10),
 (6, 11),
 (7, 12),
 (8, 13),
 (9, 14),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (10, 15),
 (11, 16),
 (12, 17),
 (13, 18),
 (14, 19),
 (15, 16),
 (16, 17),
 (17, 18),
 (18, 19),
 (15, 20),
 (16, 21),
 (17, 22),
 (18, 23),
 (19, 24),
 (20, 21),
 (21, 22),
 (22, 23),
 (23, 24)]

In [32]:
len(optmodel.arcs)

40

In [33]:
optmodel.grid

(5, 5)

### 2.2 Build Model with GurobiPy and Linear Programming

``optModel`` also allows users to employ optimization modeling languages such as GurobiPy and Pyomo. For example, with ``optGrbModel``, users can easily use Gurobi to create an LP model via overwriting ``_getModel``. Similarly, ``optGrbModel`` support Pyomo.

In [2]:
import gurobipy as gp
from gurobipy import GRB
from pyepo.model.grb import optGrbModel

class myShortestPathModel2(optGrbModel):

    def __init__(self, grid):
        """
        Args:
            grid (tuple of int): size of grid network
        """
        self.grid = grid
        self.arcs = self._getArcs()
        super().__init__()

    def _getModel(self):
        """
        A method to build Gurobi model

        Returns:
            tuple: optimization model and variables
        """
        # ceate a model
        m = gp.Model("shortest path")
        # varibles
        x = m.addVars(self.arcs, name="x")
        # sense
        m.modelSense = GRB.MINIMIZE
        # constraints
        for i in range(self.grid[0]):
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                expr = 0
                for e in self.arcs:
                    # flow in
                    if v == e[1]:
                        expr += x[e]
                    # flow out
                    elif v == e[0]:
                        expr -= x[e]
                # source
                if i == 0 and j == 0:
                    m.addConstr(expr == -1)
                # sink
                elif i == self.grid[0] - 1 and j == self.grid[0] - 1:
                    m.addConstr(expr == 1)
                # transition
                else:
                    m.addConstr(expr == 0)
        return m, x


    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges in columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

In [3]:
# solve model
optmodel = myShortestPathModel2(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Restricted license - for non-production use only - expires 2026-11-23
Obj: 2.2869939003139734
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


In [6]:
sol

[1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [4]:
m,x = optmodel._getModel()
m

<gurobi.Model Continuous instance shortest path: 0 constrs, 0 vars, No parameter changes>

In [5]:
x

{(0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 4): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (3, 8): <gurobi.Var *Awaiting Model Update*>,
 (4, 9): <gurobi.Var *Awaiting Model Update*>,
 (5, 6): <gurobi.Var *Awaiting Model Update*>,
 (6, 7): <gurobi.Var *Awaiting Model Update*>,
 (7, 8): <gurobi.Var *Awaiting Model Update*>,
 (8, 9): <gurobi.Var *Awaiting Model Update*>,
 (5, 10): <gurobi.Var *Awaiting Model Update*>,
 (6, 11): <gurobi.Var *Awaiting Model Update*>,
 (7, 12): <gurobi.Var *Awaiting Model Update*>,
 (8, 13): <gurobi.Var *Awaiting Model Update*>,
 (9, 14): <gurobi.Var *Awaiting Model Update*>,
 (10, 11): <gurobi.Var *Awaiting Model Update*>,
 (11, 12): <gurobi.Var *Awaiting Model Update*>,
 (12, 13): <gurobi.Var *Awaiting Model Update*>,
 (

### 2.3  Pre-defined Gurobi Model

PyEPO contains several pre-defined optimization models with GurobiPy and Pyomo.

In [6]:
# shortest path on the grid network
from pyepo.model.grb import shortestPathModel

In [7]:
# solve model
optmodel = shortestPathModel(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Restricted license - for non-production use only - expires 2026-11-23
Obj: 2.2869939003139734
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


### 2.4 Pre-defined MPAX Model
PyEPO contains several pre-defined optimization models with MPAX.

In [7]:
# shortest path on the grid network
from pyepo.model.mpax import shortestPathModel

In [8]:
# solve model
optmodel = shortestPathModel(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

AttributeError: 'list' object has no attribute 'shape'

## 3 Introducation to optMpaxModel

In this section, we introduce how to model and solve optimization problems in **PyEPO** using [`MPAX`](https://github.com/MIT-Lu-Lab/MPAX), a PDHG (Primal-Dual Hybrid Gradient) optimization framework.  

`optMpaxModel` is a PyEPO optimization model that leverages `MPAX `for solving **LP problems** (**not ILP**) using the PDHG algorithm. Unlike traditional solvers, MPAX is highly efficient on GPU, making it an excellent choice for large-scale decision-focused learning tasks.



### 3.1 Mathematical Formulation

The model solves a linear program (LP) of the form:

$$
\min_{\mathbf{x}} \mathbf{c}^\top \mathbf{x}
$$

Subject to:  

- Equality constraints: $\mathbf{A} \mathbf{x} = \mathbf{b} $  
- Inequality constraints: $ \mathbf{G} \mathbf{x} \geq \mathbf{h} $  
- Variable bounds: $ \mathbf{l} \leq \mathbf{x} \leq \mathbf{u} $  

Where:  
- $ c $ is the cost vector (objective function coefficients)  
- $ A, b $ define the equality constraints
- $ G, h $ define the inequality constraints
- $ l, u $ define the variable bounds

### 3.2 Model Parameters

`optMpaxModel` only accepts model coefficients in matrix and vector form. This means:
- Constraints must be explicitly provided as matrices and vectors (no symbolic constraints).
- No direct support for constraints in Python functions (e.g., defining Ax = b in a callable function).
- If `A` and `b` are not provided, it means **there are no equality constraints** in the problem.
- If `G` and `h` are not provided, it means **there are no inequality constraints** in the problem.
- The default value for `l` (lower bound) is `0`, meaning all variables are **non-negative** unless specified otherwise.
- The default value for `u` (upper bound) is infinity, meaning variables are unbounded unless explicitly set.

In [9]:
import numpy as np
import torch
# random seed for reproducibility
np.random.seed(42)
# number of variables
num_vars = 10
# random equality and inequality constraints
A = 2*np.random.rand(3, num_vars)-1  # 3 equality constraints
b = np.random.rand(3)
G = np.random.rand(5, num_vars)  # 5 inequality constraints
h = np.random.rand(5)

In [10]:
A

array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272,
        -0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007,
        -0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172],
       [ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003,
         0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917]])

In [11]:
G

array([[0.94888554, 0.96563203, 0.80839735, 0.30461377, 0.09767211,
        0.68423303, 0.44015249, 0.12203823, 0.49517691, 0.03438852],
       [0.9093204 , 0.25877998, 0.66252228, 0.31171108, 0.52006802,
        0.54671028, 0.18485446, 0.96958463, 0.77513282, 0.93949894],
       [0.89482735, 0.59789998, 0.92187424, 0.0884925 , 0.19598286,
        0.04522729, 0.32533033, 0.38867729, 0.27134903, 0.82873751],
       [0.35675333, 0.28093451, 0.54269608, 0.14092422, 0.80219698,
        0.07455064, 0.98688694, 0.77224477, 0.19871568, 0.00552212],
       [0.81546143, 0.70685734, 0.72900717, 0.77127035, 0.07404465,
        0.35846573, 0.11586906, 0.86310343, 0.62329813, 0.33089802]])

### 3.3 Create Model and Solve

Now, we create an optMpaxModel instance with the defined constraints.

In addition to model parameters, we also have two additional settings:  
- **`use_sparse_matrix`** (`bool`, default: `True`) – Stores constraint matrices in a sparse format or not. Choosing the right matrix type is crucial for optimization efficiency.  
- **`minimize`** (`bool`, default: `True`) – Defines whether to minimize the objective function.  

In [16]:
from pyepo.model.mpax import optMpaxModel
optmodel = optMpaxModel(A=A, b=b, G=G, h=h, use_sparse_matrix=False, minimize=True)

In [17]:
# generate a random cost vector
cost = np.random.rand(num_vars)
# solve the model
optmodel.setObj(cost)
sol, obj = optmodel.solve()
# print results
print(f"Objective Value: {obj}")
print(f"Solution: {sol}")

Objective Value: 0.9333382844924927
Solution: tensor([0.0000, 0.0000, 0.8207, 0.0000, 0.0000, 0.6655, 0.0000, 0.9349, 0.0000,
        0.0000])


In [13]:
cost

array([0.88721274, 0.47221493, 0.11959425, 0.71324479, 0.76078505,
       0.5612772 , 0.77096718, 0.4937956 , 0.52273283, 0.42754102])